<a href="https://colab.research.google.com/github/hamagami/anomaly-detection/blob/main/05_01_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple anomaly detection based on image features extracted by VGG16 network.



Prepare images of various guitars, and images that are not similar to guitars. Can you find an image that is not a guitar?

In [ ]:
# Download the guitar images. It'll take a while.
!wget https://dl.dropbox.com/s/dmjzsrqa9s22joi/imgs.zip
!unzip -d . imgs.zip

In [13]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Activation, Flatten
from keras import models, optimizers, layers
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
#from keras.applications.resnet50 import preprocess_input
from keras.applications.vgg16 import preprocess_input
from sklearn.svm import OneClassSVM

import glob  
from PIL import Image 
import numpy as np  
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.image as mpimg
import random

## Adjust the size of the image

In [7]:
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

## Bring up the generic learned network VGG16.

In [ ]:
model = VGG16(weights='imagenet', include_top=False, pooling="avg")

## Prepare a function to extract features from an image

In [14]:
def feature_extraction(model, img_path, image_size=224):
    im=load_img(f)
    img = expand2square(im, (128, 128, 128)).resize((image_size, image_size))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)  # add a dimention of samples
    x = preprocess_input(x)  # RGB 2 BGR and zero-centering by mean pixel based on the position of channels

    feat = model.predict(x)  # Get image features
    feat = feat.flatten()  # Convert 3-dimentional matrix to (1, n) array

    return feat

## Feature Extraction of Guitar Image

In [ ]:

files = glob.glob("./imgs/guitar/*") 
# test
for f in random.sample(files, 3):
  print(f)
  ret= feature_extraction(model, f, 224)
  plt.imshow(mpimg.imread(f))
  plt.plot(ret*10)
  plt.show()

In [ ]:
# Perform feature extraction of all guitar images. It takes quite some time.
data=[]
for f in files:
        print(f)
        ret= feature_extraction(model, f, 224)
        data.append(ret)

In [ ]:
# Let's use PCA to look at the approximate distribution.
data = np.array(data)
pca = PCA(n_components=2)
pdata =np.array(pca.fit_transform(data))
plt.scatter(pdata.T[0],pdata.T[1])
plt.title("feafure map PCA 2D")
plt.show()

In [ ]:
# oneclassSVM
mmscaler = preprocessing.MinMaxScaler() 
mmscaler.fit(data)  
sdata=mmscaler.transform(data)

#decision value
ocsvm = OneClassSVM(nu=0.001, kernel="rbf", gamma='auto')
ocsvm.fit(sdata)
y_pred = ocsvm.decision_function(sdata).ravel()
plt.plot(y_pred)
plt.title("decision value (train data)")
plt.show()

### Let's also look for features in images that are not guitars.


In [ ]:
adata=[]
files = glob.glob("./imgs/others/*")   

for f in np.sort(files):
        print(f)
        ret= feature_extraction(model, f, 224)
        plt.imshow(mpimg.imread(f))
        plt.plot(ret*10)
        plt.show()
        adata.append(ret)
adata = np.array(adata)
sadata=mmscaler.transform(adata)
apred=ocsvm.predict(sadata)
a_pca_data =np.array(pca.fit_transform(adata))

## Let's look at the difference between a non-guitar image and a guitar image on the PCA plane.It is not well represented on the two-dimensional plane, but we can see that it is still out of the guitar cluster.




In [ ]:
plt.scatter(pdata.T[0],pdata.T[1])
plt.scatter(a_pca_data.T[0][0],a_pca_data.T[1][0],label="hyotan",marker="D")
plt.scatter(a_pca_data.T[0][1],a_pca_data.T[1][1],label="neko",marker="D")
plt.scatter(a_pca_data.T[0][2],a_pca_data.T[1][2],label="chell",marker="D")
plt.scatter(a_pca_data.T[0][3],a_pca_data.T[1][3],label="ukurere",marker="D")
plt.scatter(a_pca_data.T[0][4],a_pca_data.T[1][4],label="elec",marker="D")
plt.scatter(a_pca_data.T[0][5],a_pca_data.T[1][5],label="violin",marker="D")
plt.scatter(a_pca_data.T[0][6],a_pca_data.T[1][6],label="ilust",marker="D")
plt.scatter(a_pca_data.T[0][7],a_pca_data.T[1][7],label="piano",marker="D")
plt.legend()
plt.show()